In [1]:
%defaultDatasource jdbc:h2:mem:db

# Modelagem

## Modelo Conceitual
![Modelo Conceitual](../img/conceitual.png "Title")

## Modelo Lógico

- **Disease**(*name*, occurrences)
- **Symptom**(*name*, occurrences)
- **Cause**(*disease*, *symptom*, occurrences, score)
    - Chave Estrangeira: *disease* -> **Disease**
    - Chave Estrangeira: *symptom* -> **Symptom**
- **Similarity**(disease_from, disease_to, score)
    - Chave Estrangeira: *disease_from* -> **Disease**
    - Chave Estrangeira: *disease_to* -> **Disease**


# Criação das Tabelas Relacionais

In [2]:
DROP TABLE IF EXISTS Disease;
DROP TABLE IF EXISTS Symptom;
DROP TABLE IF EXISTS Cause;
DROP TABLE IF EXISTS Similarity;

CREATE TABLE Disease (
    name VARCHAR(512) NOT NULL,
    occurrences INT NOT NULL,
    PRIMARY KEY (name)
) AS SELECT * FROM CSVREAD('../data/ncomms-disease.csv', null, 'charset=UTF-8 fieldSeparator=|');

CREATE TABLE Symptom (
    name VARCHAR(512) NOT NULL,
    occurrences INT NOT NULL,
    PRIMARY KEY (name)
) AS SELECT * FROM CSVREAD('../data/ncomms-symptom.csv', null, 'charset=UTF-8 fieldSeparator=|');

CREATE TABLE Cause (
    symptom VARCHAR(512) NOT NULL,
    disease VARCHAR(512) NOT NULL,
    occurrences INT NOT NULL,
    score DOUBLE NOT NULL,
    PRIMARY KEY (disease, symptom),
    FOREIGN KEY (disease)
        REFERENCES Disease(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (symptom)
        REFERENCES Symptom(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
) AS SELECT * FROM CSVREAD('../data/ncomms-cause.csv', null, 'charset=UTF-8 fieldSeparator=|');

CREATE TABLE Similarity (
    disease_from VARCHAR(512) NOT NULL,
    disease_to VARCHAR(512) NOT NULL,
    score DOUBLE NOT NULL,
    PRIMARY KEY (disease_from, disease_to),
    FOREIGN KEY (disease_from)
        REFERENCES Disease(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (disease_to)
        REFERENCES Disease(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
) AS SELECT * FROM CSVREAD('../data/ncomms-similarity.csv', null, 'charset=UTF-8 fieldSeparator=|');

In [3]:
SELECT * FROM Disease;
SELECT * FROM Symptom;

# Análise de Dados

### Retornar o Score médio de sintomas de cada doença

In [8]:
DROP VIEW AvgScore;

In [9]:
CREATE VIEW AvgScore AS
SELECT c.disease disease, AVG(c.score) avg
FROM Cause c
GROUP BY c.disease;

In [10]:
SELECT * FROM AvgScore;

### Sintomas com score acima da média para uma doença específica

Retornar os sintomas mais recorrentes (acima da média de score) da doença "Diabetes Mellitus, Type 1".

In [18]:
SELECT c.symptom, c.score
FROM Cause c
WHERE c.score > (SELECT a.avg FROM AvgScore a WHERE a.disease = c.disease)
    AND c.disease = 'Diabetes Mellitus, Type 1'
ORDER BY c.score DESC;

### Dado uma lista de sintomas, retornar as doenças possíveis que contém estes sintomas

- Criar uma tabela fictícia com a lista de sintomas

In [49]:
DROP VIEW SymptomsList;

In [50]:
CREATE VIEW SymptomsList AS
SELECT c.symptom
FROM Cause c
WHERE c.score > (SELECT a.avg FROM AvgScore a WHERE a.disease = c.disease)
    AND c.disease = 'Diabetes Mellitus, Type 1'
ORDER BY c.score DESC
LIMIT 4;

In [51]:
SELECT * FROM SymptomsList;

- Procurar as doenças que possuem todos os sintomas da VIEW **SymptomsList**

In [52]:
SELECT d.name, d.occurrences
FROM Disease d, Cause c, SymptomsList sl
WHERE d.name = c.disease
    AND c.symptom = sl.symptom
GROUP BY d.name
HAVING COUNT(c.symptom) = (SELECT COUNT(*) FROM SymptomsList)
ORDER BY d.occurrences DESC

### Retornar doenças similares (score > 0.8) a uma doença específica

In [70]:
SELECT disease_from, disease_to, score
FROM Similarity
WHERE (disease_from = 'Diabetes Mellitus, Type 1'
    OR disease_to = 'Diabetes Mellitus, Type 1')
    AND score > 0.8
ORDER BY score DESC

### Comparação entre sintomas de uma doença e da doença mais similar a esta

- Listar os sintomas de uma doença

In [75]:
SELECT symptom
FROM Cause
WHERE disease = 'Diabetes Mellitus, Type 1'
ORDER BY symptom

- Listar os sintomas da doença mais similar

In [76]:
SELECT symptom
FROM Cause
WHERE disease = (SELECT disease_from FROM Similarity WHERE disease_to = 'Diabetes Mellitus, Type 1' ORDER BY score DESC LIMIT 1)
ORDER BY symptom

- Listar os sintomas que fazem parte da 'Diabetes Mellitus, Type 1' e não fazem parte da doença mais similar

In [77]:
SELECT cdiabetes.symptom
FROM Cause cdiabetes
WHERE cdiabetes.disease = 'Diabetes Mellitus, Type 1'
    AND cdiabetes.symptom NOT IN (SELECT csimilar.symptom
                                  FROM Cause csimilar
                                  WHERE csimilar.disease = (SELECT disease_from FROM Similarity WHERE disease_to = 'Diabetes Mellitus, Type 1' ORDER BY score DESC LIMIT 1))
ORDER BY cdiabetes.symptom

- Listar os sintomas que fazem parte da 'Diabetes Mellitus, Type 1' e fazem parte da doença mais similar

In [78]:
SELECT cdiabetes.symptom
FROM Cause cdiabetes
WHERE cdiabetes.disease = 'Diabetes Mellitus, Type 1'
    AND cdiabetes.symptom IN (SELECT csimilar.symptom
                                  FROM Cause csimilar
                                  WHERE csimilar.disease = (SELECT disease_from FROM Similarity WHERE disease_to = 'Diabetes Mellitus, Type 1' ORDER BY score DESC LIMIT 1))
ORDER BY cdiabetes.symptom